## 80. ID番号への変換
1. ~~単語をIDに変換する辞書を作成・保存(重いため,2回目以降はセル2を実行)~~
2. 辞書の読み込み
3. 単語列をID列に変換する関数

In [1]:
# pklファイルから読み込み
import pickle

with open('../data/ch09/name_to_id.pkl', 'rb') as tf:
    name_to_id = pickle.load(tf)

In [2]:
# 与えられた単語列に対し, ID番号の列を返す関数
# ch06ではCountVectorizerを利用したため, 今回もCountVectorizerを活用する
from sklearn.feature_extraction.text import CountVectorizer

def convert_words_to_ids(words):
    '''
    input :words(単語列)
    output:ids(ID番号列)
    '''
    # analyzer: 単語列に前処理を加え, listに変換する関数
    analyzer = CountVectorizer().build_analyzer()
    word_list = analyzer(words)
    
    ids = []
    for word in word_list:
        if word in name_to_id:
            ids.append(name_to_id[word])
        else:
            ids.append(0)  # 未知語の場合, IDを0とする
    
    return ids

## GPU prepare
1. 使用可能GPUの確認
2. GPUの指定
3. PyTorchで利用できるGPU数の確認

In [3]:
# 使用可能GPUの確認
!nvidia-smi

Wed Jun 15 14:34:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:01:00.0 Off |                  Off |
| 30%   30C    P8    17W / 300W |   2181MiB / 48685MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:25:00.0 Off |                  Off |
| 31%   

In [4]:
# GPUの指定
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' #0番を使用するとき

In [5]:
# 確認
import torch
print(torch.cuda.device_count()) #Pytorchで使用できるGPU数を取得

1


## prepare
1. 語彙数の取得
2. 学習データの用意(ラベル)
3. 学習データの用意(特徴量)
4. 乱数の種を固定

In [6]:
# 語彙数の取得(ID:0の単語はまとめて1語とする), 未知語, paddingを考慮
vocab_size = max(name_to_id.values())+2

In [7]:
# 訓練・検証・評価データの用意
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# ラベル: ch08の出力を利用
Y_train = np.loadtxt('../data/ch08/Y_train.txt')
Y_valid = np.loadtxt('../data/ch08/Y_valid.txt')
Y_test = np.loadtxt('../data/ch08/Y_test.txt')

# pytorch用に変換
Y_train_long = torch.tensor(Y_train, dtype=torch.int64)
Y_valid_long = torch.tensor(Y_valid, dtype=torch.int64)
Y_test_long = torch.tensor(Y_test, dtype=torch.int64)

In [8]:
# 特徴量: convert_words_to_ids(80)を利用
def convert_text_to_features(fname):
    '''
    input :fname
    output:features(tensor)
    '''
    with open(fname, encoding='utf-8') as f:
        lines = f.readlines()
    
    # id列(list)のリストに変換
    ids_list = [convert_words_to_ids(line) for line in lines]
    
    # id列(tensor)のリストに変換
    ids_tensor = [torch.tensor(ids, dtype=torch.int64) for ids in ids_list]
    
    # 最大のid+1(vocab_size-1)でパディング
    features = torch.nn.utils.rnn.pad_sequence(ids_tensor, batch_first=True, padding_value=vocab_size-1)
    
    return features

# 特徴量抽出
X_train_long = convert_text_to_features('../data/ch06/train.txt')
X_valid_long = convert_text_to_features('../data/ch06/valid.txt')
X_test_long = convert_text_to_features('../data/ch06/test.txt')

In [9]:
# 乱数シードの固定
import random

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

## 86. 畳み込みニューラルネットワーク (CNN)
1. CNNの定義
2. ~~CNNの予測~~

In [10]:
# CNNの定義
class CNN1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_size, hidden_size, kernel_size, batch_size):
        # 層の定義
        super().__init__()
        self.batch_size = batch_size
        
        self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab_size-1)
        self.conv = nn.Conv2d(1, hidden_size, kernel_size=(kernel_size, embedding_dim), padding=(1,0))
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)

        # GPUに移す
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self = self.to(self.device)
    
    def forward(self, x):
        # embedding
        emb_out = self.emb(x)          # batch_size, sentence_length, emb_dim

        # conv
        conv_in = emb_out.unsqueeze(1) # batch_size, input_channels=1, sentence_length, emb_dim
        conv_out = self.conv(conv_in)  # batch_size, output_channels, sentence_length+2*pad-kernel+1, 1

        # relu
        relu_in = conv_out.squeeze(3)  # batch_size, output_channels, sentence_length+2*pad-kernel+1, 1
        relu_out = self.relu(relu_in)  # batch_size, output_channels, sentence_length+2*pad-kernel+1

        # pool
        pool_out = F.max_pool2d(relu_out, kernel_size=(1, relu_out.size(2))) # batch_size, output_channels, 1

        # fc
        fc_in = pool_out.squeeze(2)    # batch_size, output_channels
        fc_out = self.fc(fc_in)        # batch_size, output_size
        
        return fc_out

## 87. 確率的勾配降下法によるCNNの学習
1. GPUに対応したaccuracy計測関数の定義(CNN)
2. CNNの学習

In [11]:
# accuracyの計測
def measure_loss_accuracy3(model, criterion, dataloader):
    '''
    input : model, criterion, dataloader
    output: loss, accuracy
    '''
    running_loss = 0
    correct_num = 0
    device = model.device
    for X, Y in dataloader:
        # GPU上に
        X = X.to(device)
        Y = Y.to(device)
        predict_y = model.forward(X)
        
        # lossの計算
        loss = criterion(predict_y, Y)
        running_loss += loss.item()
        
        # accuracyの計算
        predict_label = torch.max(predict_y, 1)[1]
        for i in range(batch_size):
            if predict_label[i] == Y[i]:
                correct_num += 1
    
    loss_avg = running_loss/len(dataloader)
    accuracy = correct_num/(len(dataloader)*batch_size)
    
    return loss_avg, accuracy

In [12]:
# CNNの学習
import torch.optim as optim

# ハイパーパラメータ
embedding_dim = 300
hidden_size = 50
kernel_size = 3
batch_size = 16
max_epoch = 10
lr = 0.002

# dataloaderの定義
train_dataset2 = torch.utils.data.TensorDataset(X_train_long, Y_train_long)
train_dataloader2 = torch.utils.data.DataLoader(train_dataset2, batch_size=batch_size, drop_last=True)

valid_dataset2 = torch.utils.data.TensorDataset(X_valid_long, Y_valid_long)
valid_dataloader2 = torch.utils.data.DataLoader(valid_dataset2, batch_size=batch_size, drop_last=True)

# モデルの定義
fix_seed(42)
cnn2 = CNN1(vocab_size, embedding_dim, 4, hidden_size, kernel_size, batch_size)
criterion5 = nn.CrossEntropyLoss()
optimizer5 = optim.SGD(cnn2.parameters(), lr)
device = cnn2.device

# SGDによる学習
for epoch in range(max_epoch):
    running_loss = 0
    for X, Y in train_dataloader2:
        # GPU上に
        X = X.to(device)
        Y = Y.to(device)
        # forward
        predict_y = cnn2.forward(X)
        loss = criterion5(predict_y, Y)

        # backward
        optimizer5.zero_grad()
        loss.backward()

        # 更新
        optimizer5.step()
        running_loss += loss.item()

    # loss, accuracyの表示
    valid_loss, valid_acc = measure_loss_accuracy3(cnn2, criterion5, valid_dataloader2)
    train_loss, train_acc = measure_loss_accuracy3(cnn2, criterion5, train_dataloader2)
    train_loss = running_loss/len(train_dataloader2)
    
    print(f'epoch: {epoch}')
    print(f'   train loss: {train_loss}\ttrain acc: {train_acc}')
    print(f'   valid loss: {valid_loss}\tvalid acc: {valid_acc}')

epoch: 0
   train loss: 1.1003955877881715	train acc: 0.6557346326836582
   valid loss: 1.0100657817829086	valid acc: 0.6528614457831325
epoch: 1
   train loss: 0.9351205953593733	train acc: 0.7057721139430285
   valid loss: 0.909128091421472	valid acc: 0.6867469879518072
epoch: 2
   train loss: 0.8300766886799767	train acc: 0.7350074962518741
   valid loss: 0.8365564554570669	valid acc: 0.7070783132530121
epoch: 3
   train loss: 0.7446469958128779	train acc: 0.7563718140929535
   valid loss: 0.779061424444957	valid acc: 0.7183734939759037
epoch: 4
   train loss: 0.6711190701245904	train acc: 0.7797038980509745
   valid loss: 0.7314856174480484	valid acc: 0.7304216867469879
epoch: 5
   train loss: 0.6056803272760611	train acc: 0.8045352323838081
   valid loss: 0.6925127603203417	valid acc: 0.7394578313253012
epoch: 6
   train loss: 0.5464184639201887	train acc: 0.8320839580209896
   valid loss: 0.6590889484049326	valid acc: 0.7530120481927711
epoch: 7
   train loss: 0.4923038481548987	